In [393]:
# HW4.2 ACADEMY OF PY

import os
import numpy as np
import pandas as pd

# Pull in the files and review variables

schools = "/Users/frimpter/Documents/data_science/ru_datascience_bootcamp/Homework4 Pandas/schools_complete.csv"
students = "/Users/frimpter/Documents/data_science/ru_datascience_bootcamp/Homework4 Pandas/students_complete.csv"

schools_df = pd.DataFrame(pd.read_csv(schools))
students_df = pd.DataFrame(pd.read_csv(students))

schools_df = schools_df.rename(columns={"name":"School", "type":"School Type", "size":"Total Students", "budget":"Total Budget"})
students_df = students_df.rename(columns={"school":"School", "grade":"Grade"})
#schools_df.head()
#students_df.head()

In [394]:
# DISTRICT SUMMARY

# Calculate number of students passing math, reading and BOTH
mathpass = students_df.loc[students_df["math_score"] >= 70, :] # Students with passing math scores
readpass = students_df.loc[students_df["reading_score"] >= 70, :] # Students with passing reading scores
totalpass = students_df.loc[(students_df["math_score"] >= 70) & (students_df["reading_score"] >= 70), :] # Students passing both math and reading

# Create a Summary Table/df of district-level information
districts_df = pd.DataFrame({
    "Total Schools": [schools_df["School"].count()],
    "Total Students": [schools_df["Total Students"].sum()],
    "Total Budget": [schools_df["Total Budget"].sum()],
    "Average Math Score": [students_df["math_score"].mean()],
    "Passing Math %": [mathpass["math_score"].count() / students_df["name"].count()],
    "Average Reading Score": [students_df["reading_score"].mean()],
    "Passing Reading %": [readpass["reading_score"].count() / students_df["name"].count()],
    "Overall Passing %": [totalpass["Student ID"].count() / students_df["name"].count()]
})

# Format table
districts_df = districts_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Passing Math %", "Average Reading Score", "Passing Reading %", "Overall Passing %"]]
districts_df["Total Students"] = districts_df["Total Students"].map("{:,}".format)
districts_df["Total Budget"] = districts_df["Total Budget"].map("${:,}".format)
districts_df["Average Math Score"] = districts_df["Average Math Score"].map("{:,.1f}".format)
districts_df["Passing Math %"] = districts_df["Passing Math %"].map("{:.1%}".format)
districts_df["Average Reading Score"] = districts_df["Average Reading Score"].map("{:,.1f}".format)
districts_df["Passing Reading %"] = districts_df["Passing Reading %"].map("{:.1%}".format)
districts_df["Overall Passing %"] = districts_df["Overall Passing %"].map("{:.1%}".format)

districts_df

,Total Schools,Total Students,Total Budget,Average Math Score,Passing Math %,Average Reading Score,Passing Reading %,Overall Passing %
0,15,"39,170","$24,649,428",79.0,75.0%,81.9,85.8%,65.2%


In [395]:
# SCHOOL SUMMARY
# Create a Summary Table of school-level information

# Prepare the schools data to merge by index/School
schools_df = schools_df.sort_values(["School"])
schools_df.set_index("School", inplace=True)
schools_df = schools_df[["School Type", "Total Students", "Total Budget"]]
#schools_df

# Prepare students data to merge by index/School with mean values
stugroup = students_df.groupby(["School"])
scoresgroup = pd.DataFrame(stugroup.mean())

# New DataFrame
combo_df = pd.merge(schools_df, scoresgroup, right_index=True, left_index=True)
#combo_df

In [396]:
# Format Columns
combo_df = combo_df.rename(columns={
    "reading_score":"Average Reading Score",
    "math_score":"Average Math Score"
})

# Append computed parameters
# Budget Per Student
combo_df["Total Students"] = pd.to_numeric(combo_df["Total Students"])
combo_df["Total Budget"] = pd.to_numeric(combo_df["Total Budget"])
combo_df["Budget Per Student"] = combo_df["Total Budget"] / combo_df["Total Students"]

# Average Math, Reading, Overall Passing scores per school
MathPass = mathpass.groupby(["School"])
combo_df["% Passing Math"] = (MathPass["Student ID"].count() / combo_df["Total Students"])*100
ReadPass = readpass.groupby(["School"])
combo_df["% Passing Reading"] = (ReadPass["Student ID"].count() / combo_df["Total Students"])*100
TotalPass = totalpass.groupby(["School"])
combo_df["% Passing Overall"] = (TotalPass["Student ID"].count() / combo_df["Total Students"])*100

# Arrange table columns
combo_df = combo_df[["School Type", "Total Students", "Total Budget", "Budget Per Student", "Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

# Format values
combo_df["Total Students"] = combo_df["Total Students"].map("{:,}".format)
combo_df["Total Budget"] = combo_df["Total Budget"].map("${:,}".format)
combo_df["Budget Per Student"] = combo_df["Budget Per Student"].map("${:,.0f}".format)
combo_df["Average Math Score"] = combo_df["Average Math Score"].map("{:.1f}".format)
combo_df["% Passing Math"] = combo_df["% Passing Math"].map("{:.1f}%".format)
combo_df["Average Reading Score"] = combo_df["Average Reading Score"].map("{:.1f}".format)
combo_df["% Passing Reading"] = combo_df["% Passing Reading"].map("{:.1f}%".format)
combo_df["% Passing Overall"] = combo_df["% Passing Overall"].map("{:.1f}%".format)

combo_df

,School Type,Total Students,Total Budget,Budget Per Student,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Passing Overall
School,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628,77.0,66.7%,81.0,81.9%,54.6%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,94.1%,84.0,97.0%,91.3%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,66.0%,81.2,80.7%,53.2%
Ford High School,District,"2,739","$1,763,916",$644,77.1,68.3%,80.7,79.3%,54.3%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,93.4%,83.8,97.1%,90.6%
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,66.8%,80.9,80.9%,53.5%
Holden High School,Charter,427,"$248,087",$581,83.8,92.5%,83.8,96.3%,89.2%
Huang High School,District,"2,917","$1,910,635",$655,76.6,65.7%,81.2,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,66.1%,81.0,81.2%,53.5%


In [397]:
# TOP 5 PERFORMERS
top_five = combo_df.sort_values("% Passing Overall", ascending=False)
top_five.head()

,School Type,Total Students,Total Budget,Budget Per Student,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Passing Overall
School,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,94.1%,84.0,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,93.3%,83.8,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,93.4%,83.8,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,93.9%,84.0,96.5%,90.6%
Pena High School,Charter,962,"$585,858",$609,83.8,94.6%,84.0,95.9%,90.5%


In [398]:
# WORST 5 PERFORMERS
top_five.tail()

,School Type,Total Students,Total Budget,Budget Per Student,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Passing Overall
School,,,,,,,,,
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,66.8%,80.9,80.9%,53.5%
Huang High School,District,"2,917","$1,910,635",$655,76.6,65.7%,81.2,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,66.1%,81.0,81.2%,53.5%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,66.0%,81.2,80.7%,53.2%
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,66.4%,80.7,80.2%,53.0%


In [399]:
# MATH SCORES BY GRADE AND SCHOOL
schoolgrade = students_df.groupby(["School", "Grade"]).mean().unstack()
SchoolGrade = pd.DataFrame(schoolgrade)
SchoolGrade = SchoolGrade.rename(columns={"math_score":"Average Math Score", "reading_score":"Average Reading Score"})
MathGrade = SchoolGrade[["Average Math Score"]]

MathGrade

Average Math Score                                 
Grade                               10th       11th       12th        9th
School                                                                   
Bailey High School             76.996772  77.515588  76.492218  77.083676
Cabrera High School            83.154506  82.765560  83.277487  83.094697
Figueroa High School           76.539974  76.884344  77.151369  76.403037
Ford High School               77.672316  76.918058  76.179963  77.361345
Griffin High School            84.229064  83.842105  83.356164  82.044010
Hernandez High School          77.337408  77.136029  77.186567  77.438495
Holden High School             83.429825  85.000000  82.855422  83.787402
Huang High School              75.908735  76.446602  77.225641  77.027251
Johnson High School            76.691117  77.491653  76.863248  77.187857
Pena High School               83.372000  84.328125  84.121547  83.625455
Rodriguez High School          76.612500  76.395626  77.690748  76.859966
Shelton High School            82.917411  83.383495  83.778976  83.420755
Thomas High School             83.087886  83.498795  83.497041  83.590022
Wilson High School             83.724422  83.195326  83.035794  83.085578
Wright High School             84.010288  83.836782  83.644986  83.264706

In [400]:
# READING SCORES BY GRADE AND SCHOOL
ReadGrade = SchoolGrade[["Average Reading Score"]]

ReadGrade

Average Reading Score                                 
Grade                                  10th       11th       12th        9th
School                                                                      
Bailey High School                80.907183  80.945643  80.912451  81.303155
Cabrera High School               84.253219  83.788382  84.287958  83.676136
Figueroa High School              81.408912  80.640339  81.384863  81.198598
Ford High School                  81.262712  80.403642  80.662338  80.632653
Griffin High School               83.706897  84.288089  84.013699  83.369193
Hernandez High School             80.660147  81.396140  80.857143  80.866860
Holden High School                83.324561  83.815534  84.698795  83.677165
Huang High School                 81.512386  81.417476  80.305983  81.290284
Johnson High School               80.773431  80.616027  81.227564  81.260714
Pena High School                  83.612000  84.335938  84.591160  83.807273
Rodriguez High School             80.629808  80.864811  80.376426  80.993127
Shelton High School               83.441964  84.373786  82.781671  84.122642
Thomas High School                84.254157  83.585542  83.831361  83.728850
Wilson High School                84.021452  83.764608  84.317673  83.939778
Wright High School                83.812757  84.156322  84.073171  83.833333

In [401]:
# SCORES BY SPENDING PER STUDENT

# Make a New (Unformatted) DataFrame similar to combo_df
newcombo_df = pd.merge(schools_df, scoresgroup, right_index=True, left_index=True)

# Format Columns
newcombo_df = newcombo_df.rename(columns={
    "reading_score":"Average Reading Score",
    "math_score":"Average Math Score"
})

# Spending Per Student
newcombo_df["Total Students"] = pd.to_numeric(newcombo_df["Total Students"])
newcombo_df["Total Budget"] = pd.to_numeric(newcombo_df["Total Budget"])
newcombo_df["Spending Per Student"] = newcombo_df["Total Budget"] / newcombo_df["Total Students"]

# Average Math, Reading, Overall Passing scores per school
MathPass = mathpass.groupby(["School"])
newcombo_df["% Passing Math"] = (MathPass["Student ID"].count() / newcombo_df["Total Students"])*100
ReadPass = readpass.groupby(["School"])
newcombo_df["% Passing Reading"] = (ReadPass["Student ID"].count() / newcombo_df["Total Students"])*100
TotalPass = totalpass.groupby(["School"])
newcombo_df["% Passing Overall"] = (TotalPass["Student ID"].count() / newcombo_df["Total Students"])*100

# Check range of average spending values to inform bins
# print(combo_df["Budget Per Student"].min())
# print(combo_df["Budget Per Student"].max())

# Add spending category
bins = [550, 600, 650, 700]
labels = ["\$550 to \$600", "\$601 to \$650", "\$651 to \$700"]
newcombo_df["Spending Category"] = pd.cut(newcombo_df["Spending Per Student"], bins, labels=labels)

newcombo_df = newcombo_df[["Spending Category", "Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

#newcombo_df

In [402]:
# PRESENT SCORES BY SPENDING PER STUDENT CATEGORY

spending_pivot = pd.DataFrame(newcombo_df.pivot_table(index="Spending Category", values=["Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]))
spending_pivot = spending_pivot[["Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

# Format values
spending_pivot["Average Math Score"] = spending_pivot["Average Math Score"].map("{:.1f}".format)
spending_pivot["% Passing Math"] = spending_pivot["% Passing Math"].map("{:.1f}%".format)
spending_pivot["Average Reading Score"] = spending_pivot["Average Reading Score"].map("{:.1f}".format)
spending_pivot["% Passing Reading"] = spending_pivot["% Passing Reading"].map("{:.1f}%".format)
spending_pivot["% Passing Overall"] = spending_pivot["% Passing Overall"].map("{:.1f}%".format)

spending_pivot

,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Passing Overall
Spending Category,,,,,
\$550 to \$600,83.4,93.5%,83.9,96.5%,90.3%
\$601 to \$650,79.4,76.8%,82.0,86.7%,67.6%
\$651 to \$700,77.0,66.2%,81.1,81.1%,53.5%


In [403]:
# SCORES BY SCHOOL SIZE

# Reset newcombo_df DataFrame
newcombo_df = pd.merge(schools_df, scoresgroup, right_index=True, left_index=True)

# Format Columns
newcombo_df = newcombo_df.rename(columns={
    "reading_score":"Average Reading Score",
    "math_score":"Average Math Score"
})

newcombo_df["Total Students"] = pd.to_numeric(newcombo_df["Total Students"])

# Average Math, Reading, Overall Passing scores per school
MathPass = mathpass.groupby(["School"])
newcombo_df["% Passing Math"] = (MathPass["Student ID"].count() / newcombo_df["Total Students"])*100
ReadPass = readpass.groupby(["School"])
newcombo_df["% Passing Reading"] = (ReadPass["Student ID"].count() / newcombo_df["Total Students"])*100
TotalPass = totalpass.groupby(["School"])
newcombo_df["% Passing Overall"] = (TotalPass["Student ID"].count() / newcombo_df["Total Students"])*100

# Check range of average spending values to inform bins
#print(newcombo_df["Total Students"].min())
#print(newcombo_df["Total Students"].max())

# Add size categories
bins = [0, 999, 1999, 2999, 3999, 5000]
labels = ["<1,000", "1,000 to 1,999", "2,000 to 2,999", "3,000 to 3,999", "4,000 to 5,000"]
newcombo_df["Student Population"] = pd.cut(newcombo_df["Total Students"], bins, labels=labels)

newcombo_df = newcombo_df[["Student Population", "Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

#newcombo_df

In [404]:
# PRESENT SCORES BY SCHOOL SIZE GROUPING

size_pivot = pd.DataFrame(newcombo_df.pivot_table(index="Student Population", values=["Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]))
size_pivot = size_pivot[["Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

# Format values
size_pivot["Average Math Score"] = size_pivot["Average Math Score"].map("{:.1f}".format)
size_pivot["% Passing Math"] = size_pivot["% Passing Math"].map("{:.1f}%".format)
size_pivot["Average Reading Score"] = size_pivot["Average Reading Score"].map("{:.1f}".format)
size_pivot["% Passing Reading"] = size_pivot["% Passing Reading"].map("{:.1f}%".format)
size_pivot["% Passing Overall"] = size_pivot["% Passing Overall"].map("{:.1f}%".format)

size_pivot

,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Passing Overall
Student Population,,,,,
"<1,000",83.8,93.6%,83.9,96.1%,89.9%
"1,000 to 1,999",83.4,93.6%,83.9,96.8%,90.6%
"2,000 to 2,999",78.4,73.5%,81.8,84.5%,62.9%
"3,000 to 3,999",76.8,66.4%,80.7,80.2%,53.0%
"4,000 to 5,000",77.1,66.5%,81.0,81.3%,53.9%


In [405]:
# SCORES BY SCHOOL TYPE

# Reset newcombo_df DataFrame
newcombo_df = pd.merge(schools_df, scoresgroup, right_index=True, left_index=True)

# Format Columns
newcombo_df = newcombo_df.rename(columns={
    "reading_score":"Average Reading Score",
    "math_score":"Average Math Score"
})

newcombo_df["Total Students"] = pd.to_numeric(newcombo_df["Total Students"])

# Average Math, Reading, Overall Passing scores per school
MathPass = mathpass.groupby(["School"])
newcombo_df["% Passing Math"] = (MathPass["Student ID"].count() / newcombo_df["Total Students"])*100
ReadPass = readpass.groupby(["School"])
newcombo_df["% Passing Reading"] = (ReadPass["Student ID"].count() / newcombo_df["Total Students"])*100
TotalPass = totalpass.groupby(["School"])
newcombo_df["% Passing Overall"] = (TotalPass["Student ID"].count() / newcombo_df["Total Students"])*100

newcombo_df = newcombo_df[["School Type", "Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

#newcombo_df

In [406]:
# PRESENT SCORES BY SCHOOL TYPE

type_pivot = pd.DataFrame(newcombo_df.pivot_table(index="School Type", values=["Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]))
type_pivot = type_pivot[["Average Math Score", "% Passing Math", "Average Reading Score", "% Passing Reading", "% Passing Overall"]]

# Format values
type_pivot["Average Math Score"] = type_pivot["Average Math Score"].map("{:.1f}".format)
type_pivot["% Passing Math"] = type_pivot["% Passing Math"].map("{:.1f}%".format)
type_pivot["Average Reading Score"] = type_pivot["Average Reading Score"].map("{:.1f}".format)
type_pivot["% Passing Reading"] = type_pivot["% Passing Reading"].map("{:.1f}%".format)
type_pivot["% Passing Overall"] = type_pivot["% Passing Overall"].map("{:.1f}%".format)

type_pivot

,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.5,93.6%,83.9,96.6%,90.4%
District,77.0,66.5%,81.0,80.8%,53.7%


In [422]:
print("Three Trends from School Data Analysis: ")
print("\n1. In the district, performance is better in Reading than in Math, however little more than half (65.2%) of students pass both reading and math.")
print("\n2. A greater proportion of students at Charter schools pass math, reading and both compared to District schools.")
print("\n3. These findings suggest that students at smaller schools achieve better performance in all categories, and spending is not necessarily indicative of performance.")


Three Trends from School Data Analysis: 

1. In the district, performance is better in Reading than in Math, however little more than half (65.2%) of students pass both reading and math.

2. A greater proportion of students at Charter schools pass math, reading and both compared to District schools.

3. These findings suggest that students at smaller schools achieve better performance in all categories, and spending is not necessarily indicative of performance.
